In [0]:
# Célula 1: Configuração de Caminhos e Listas
caminho_volume = "/Volumes/catalogo_energia/bronze/dados_brutos_csv/"

print(f"--> Configurando origem dos dados em: {caminho_volume}")

# Lista 1: Consumo Geral (Português + Inglês)
arquivos_consumo_geral = [
    f"{caminho_volume}consumo_energia_2023.csv",
    f"{caminho_volume}consumo_energia_2024.csv",
    f"{caminho_volume}consumo_energia2025.csv"
]

# Lista 2: Energia Limpa (Português + Inglês)
arquivos_energia_limpa = [
    f"{caminho_volume}consumo_energia_limpa_2023.csv",
    f"{caminho_volume}consumo_energia_limpa_2024.csv",
    f"{caminho_volume}consumo_energia_limpa2025.csv"
]

print("Listas de arquivos configuradas com sucesso!")

In [0]:
# Célula 2: Definição da Função de Ingestão (Versão Higienizada)
from pyspark.sql.functions import current_timestamp, col

def ingerir_para_bronze(lista_arquivos, nome_tabela_destino):
    print(f"--> Iniciando leitura e gravação para: {nome_tabela_destino}")
    
    try:
        # 1. Leitura (Raw)
        df = (spark.read
              .format("csv")
              .option("header", "true")
              .option("inferSchema", "true")
              .load(lista_arquivos))
        
        # 1.5. Higienização dos Nomes das Colunas
        # Substitui espaços por '_' e remove caracteres proibidos para o formato delta (: ,;{}()=)
        novas_colunas = []
        for coluna in df.columns:
            nova_coluna = (coluna.replace(" ", "_")
                                 .replace(",", "")
                                 .replace(";", "")
                                 .replace("{", "")
                                 .replace("}", "")
                                 .replace("(", "")
                                 .replace(")", "")
                                 .replace("\n", "")
                                 .replace("\t", "")
                                 .replace("=", ""))
            novas_colunas.append(nova_coluna)
        
        # Aplica os novos nomes ao DataFrame
        df = df.toDF(*novas_colunas)
        
        # 2. Enriquecimento (Metadados)
        df_enrich = (df
                     .withColumn("nome_arquivo_origem", col("_metadata.file_path"))
                     .withColumn("data_carga", current_timestamp()))
        
        # 3. Escrita (Delta)
        (df_enrich.write
         .format("delta")
         .mode("overwrite")
         .option("overwriteSchema", "true")
         .saveAsTable(f"catalogo_energia.bronze.{nome_tabela_destino}"))
        
        print(f"✅ Sucesso! Tabela criada: catalogo_energia.bronze.{nome_tabela_destino}")
        print(f"📊 Total de linhas: {df_enrich.count()}\n")
        
    except Exception as e:
        print(f"❌ Erro ao processar {nome_tabela_destino}: {str(e)}")

In [0]:
# Célula 3: Executando a carga
ingerir_para_bronze(arquivos_consumo_geral, "consumo_geral")
ingerir_para_bronze(arquivos_energia_limpa, "energia_limpa")